In [43]:
import numpy as np
import tensorflow
import pandas as pd
import matplotlib.pyplot as plot
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv("2011-capitalbikeshare-tripdata.csv", encoding = "big5")
df.head()
# x=df.iloc[:,:-2].values
# y=df.iloc[:,8].values


,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,5th & F St NW,31620,5th & F St NW,W00247,Member
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,14th & Harvard St NW,31101,14th & V St NW,W00675,Casual
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,Georgia & New Hampshire Ave NW,31104,Adams Mill & Columbia Rd NW,W00357,Member
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,10th & U St NW,31503,Florida Ave & R St NW,W00970,Member
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,Adams Mill & Columbia Rd NW,31106,Calvert & Biltmore St NW,W00346,Casual


In [3]:
#計算借出次數並且顯示最多的前五筆資料
station_counts_start = pd.DataFrame(df['Start station number'].value_counts())
station_counts_start.head()

,Start station number
31200,49093
31201,32884
31623,32058
31104,29964
31214,28160


In [4]:
#計算歸還次數並且顯示最多的前五筆資料
station_counts_end = pd.DataFrame(df['End station number'].value_counts())
station_counts_end.head()


,End station number
31200,53910
31201,36638
31623,32252
31214,29103
31104,26248


In [5]:
# 計算單站借出與歸還的次數
station_counts = pd.concat([station_counts_start, station_counts_end], axis=1)
station_counts['total'] = station_counts['Start station number']+station_counts['End station number']
station_counts['staion ratio'] = ((station_counts['total'])/2453534)*100
station_counts.head()


,Start station number,End station number,total,staion ratio
31000,1171,925,2096,0.085428
31001,3492,3495,6987,0.284773
31002,2883,3407,6290,0.256365
31003,3432,3532,6964,0.283835
31004,1598,1644,3242,0.132136


In [6]:
#假設取出總數大於25000的站建立index
# stations_filter = station_counts['total'] > 25000
stations = list(station_counts['total'].index)
print(stations)

[31000, 31001, 31002, 31003, 31004, 31005, 31006, 31007, 31008, 31009, 31010, 31011, 31012, 31013, 31014, 31015, 31016, 31017, 31018, 31019, 31020, 31021, 31022, 31052, 31100, 31101, 31102, 31103, 31104, 31105, 31106, 31107, 31108, 31109, 31110, 31111, 31112, 31113, 31115, 31116, 31200, 31201, 31202, 31203, 31204, 31205, 31206, 31207, 31208, 31209, 31211, 31212, 31213, 31214, 31215, 31216, 31217, 31218, 31219, 31220, 31221, 31222, 31223, 31224, 31225, 31226, 31227, 31228, 31229, 31230, 31231, 31232, 31233, 31235, 31236, 31237, 31238, 31239, 31241, 31242, 31244, 31245, 31246, 31261, 31262, 31266, 31300, 31301, 31302, 31303, 31304, 31305, 31307, 31400, 31401, 31402, 31404, 31500, 31501, 31502, 31503, 31504, 31505, 31506, 31600, 31601, 31602, 31603, 31604, 31605, 31606, 31607, 31608, 31609, 31610, 31611, 31613, 31614, 31615, 31616, 31617, 31618, 31619, 31620, 31621, 31622, 31623, 31624, 31625, 31626, 31700, 31701, 31702, 31703, 31704, 31705, 31706, 31707, 31708, 31801, 31802, 31804, 31805

In [7]:
# #從原資料中提取出list中有的資料
# df_stations = df[(df['Start station number'].isin(stations))|(df['End station number'].isin(stations))]
# df_stations.head(5)

In [8]:
#轉換start date,end date為datetime
df['Start date'] = pd.to_datetime(df['Start date'])
df['End date'] = pd.to_datetime(df['End date'])

In [9]:
#建立資料模型
df_model = df[['Start date', 'End date', 'Start station number','End station number']]
df_model.info()
df_model.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226767 entries, 0 to 1226766
Data columns (total 4 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   Start date            1226767 non-null  datetime64[ns]
 1   End date              1226767 non-null  datetime64[ns]
 2   Start station number  1226767 non-null  int64         
 3   End station number    1226767 non-null  int64         
dtypes: datetime64[ns](2), int64(2)
memory usage: 37.4 MB


,Start date,End date,Start station number,End station number
0,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620
1,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101
2,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104


In [10]:
#用Start date建立index
df_model.index = df_model['Start date']
df_model.drop('Start date',axis=1,inplace=True)
df_model.head()


C:\Users\User\Anaconda3\envs\tensetflowenv\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,End date,Start station number,End station number
Start date,,,
2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620
2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101
2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104
2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503
2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106


In [11]:
#分割出星期幾,時間的欄位
df_model['day'] = df_model.index.day_name()
df_model['year'] = df_model.index.map(lambda x:str(x)[:4])
df_model['month'] = df_model.index.map(lambda x:str(x)[5:7])
df_model['time'] = df_model.index.map(lambda x:str(x)[11:19])


C:\Users\User\Anaconda3\envs\tensetflowenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\User\Anaconda3\envs\tensetflowenv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\User\Anaconda3\envs\tensetflowenv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [12]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1226767 entries, 2011-01-01 00:01:29 to 2011-12-31 23:55:56
Data columns (total 7 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   End date              1226767 non-null  datetime64[ns]
 1   Start station number  1226767 non-null  int64         
 2   End station number    1226767 non-null  int64         
 3   day                   1226767 non-null  object        
 4   year                  1226767 non-null  object        
 5   month                 1226767 non-null  object        
 6   time                  1226767 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 74.9+ MB


In [13]:
#建立歸還列的模型(作為未來歸還預測的模型)
end = df_model[df_model['Start station number'].isin(stations)]
end.head()

,End date,Start station number,End station number,day,year,month,time
Start date,,,,,,,
2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Saturday,2011,01,00:01:29
2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Saturday,2011,01,00:02:46
2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Saturday,2011,01,00:06:13
2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Saturday,2011,01,00:09:21
2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Saturday,2011,01,00:28:26


In [14]:
end['time_col'] = end.index.map(lambda x:str(x)[:13])
end.head(1)

,End date,Start station number,End station number,day,year,month,time,time_col
Start date,,,,,,,,
2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Saturday,2011,01,00:01:29,2011-01-01 00


In [15]:
end_model = pd.DataFrame(end.groupby(['Start station number', 'time_col'])['time'].count())

In [16]:
end_model.rename(columns={'time':'end_count'}, inplace=True)
end_model.reset_index(inplace=True)
end_model.index = end_model['time_col']

In [17]:
end_model.head(10)

,Start station number,time_col,end_count
time_col,,,
2011-01-07 09,31000,2011-01-07 09,1
2011-01-07 12,31000,2011-01-07 12,1
2011-01-08 00,31000,2011-01-08 00,1
2011-01-08 14,31000,2011-01-08 14,1
2011-01-09 12,31000,2011-01-09 12,2
2011-01-10 08,31000,2011-01-10 08,1
2011-01-10 18,31000,2011-01-10 18,1
2011-01-11 07,31000,2011-01-11 07,1
2011-01-12 07,31000,2011-01-12 07,1


In [18]:
end_model['year'] = end_model.index.map(lambda x:str(x)[:4])
end_model['month'] = end_model.index.map(lambda x:str(x)[5:7])
end_model['hour'] = end_model.index.map(lambda x:str(x)[11:13])

In [19]:
end_model.head(3)

,Start station number,time_col,end_count,year,month,hour
time_col,,,,,,
2011-01-07 09,31000,2011-01-07 09,1,2011,01,09
2011-01-07 12,31000,2011-01-07 12,1,2011,01,12
2011-01-08 00,31000,2011-01-08 00,1,2011,01,00


In [20]:
end_model['time_col'] = pd.to_datetime(end_model['time_col'])
end_model.index = end_model['time_col']
end_model['day'] = end_model.index.day_name()
end_model.head(3)

,Start station number,time_col,end_count,year,month,hour,day
time_col,,,,,,,
2011-01-07 09:00:00,31000,2011-01-07 09:00:00,1,2011,01,09,Friday
2011-01-07 12:00:00,31000,2011-01-07 12:00:00,1,2011,01,12,Friday
2011-01-08 00:00:00,31000,2011-01-08 00:00:00,1,2011,01,00,Saturday


In [21]:
end_model.drop('time_col', axis=1,inplace=True)
end_model.head(1)

,Start station number,end_count,year,month,hour,day
time_col,,,,,,
2011-01-07 09:00:00,31000,1,2011,01,09,Friday


In [22]:
end_dummies = pd.get_dummies(data=end_model, columns=['Start station number','year','month','hour','day'])

In [23]:
end_dummies .head(3)

,end_count,Start station number_31000,Start station number_31001,Start station number_31002,Start station number_31003,Start station number_31004,Start station number_31005,Start station number_31006,Start station number_31007,Start station number_31008,...,hour_21,hour_22,hour_23,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
time_col,,,,,,,,,,,,,,,,,,,,,
2011-01-07 09:00:00,1,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2011-01-07 12:00:00,1,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2011-01-08 00:00:00,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [24]:
end_dummies['end_count'].skew() #非常歪斜

2.596944717537416

In [25]:
#取log接近常態分佈
log_count = np.log(end_dummies['end_count']).skew() 
log_count

0.5543780192063837

In [26]:
end_dummies['end_count(log)'] = np.log(end_dummies['end_count'])
end_dummies.head(3)

,end_count,Start station number_31000,Start station number_31001,Start station number_31002,Start station number_31003,Start station number_31004,Start station number_31005,Start station number_31006,Start station number_31007,Start station number_31008,...,hour_22,hour_23,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,end_count(log)
time_col,,,,,,,,,,,,,,,,,,,,,
2011-01-07 09:00:00,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0.0
2011-01-07 12:00:00,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0.0
2011-01-08 00:00:00,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0


In [27]:
end_dummies.shape

(405084, 190)

In [28]:
#保留20%的test data
train = end_dummies[:324067]
test = end_dummies[324067:]

In [53]:
y_train = train['end_count(log)']
x_train = train.drop(['end_count','end_count(log)'],axis=1)
y_test = test['end_count(log)']
x_test = test.drop(['end_count','end_count(log)'],axis=1)

In [40]:
x_train.shape

(324067, 188)

In [51]:
y_test.shape

(81017,)

In [42]:
#linear regression
# linear = LinearRegression()
# linear.fit(x_train,y_train)
# predicted = linear.predict(x_test)

In [49]:
rfModel = RandomForestRegressor(n_estimators=100,random_state=50,n_jobs=2)
rfModel.fit(x_train,y_train)
preds = rfModel.predict(x_train)

In [56]:
time_col=x_test.index
predsTest = rfModel.predict(x_test)
submission = pd.DataFrame({
        "datetime": time_col,
        "count": [max(0, x) for x in np.exp(predsTest)]
    })
submission.to_csv('sharebike_pred.csv', index=False)